In [14]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
import string
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
sc = pyspark.SparkContext()
from pyspark.sql import *
#install numpy if you havent already, otherwise one of the ml.featuer imports will throw an error cause it needs numpy
#%pip install numpy

from pyspark.sql import functions as F
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
spark = SparkSession.builder.appName("Projec3")\
        .config("spark.some.config.option","some-value")\
        .getOrCreate()


df = spark.read.load("/home/cse587/CSE587/Project3/train.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')


#punc = string.punctuation
def removePunctuation(s):
    print("type of s: ",s)
    return re.sub(r'[^\w\s]',"",s)
#data = re.sub(r'[^\w\s]',"",data)

udf = UserDefinedFunction(lambda x: removePunctuation(x), StringType())
df = df.select(*[udf(column).alias("plot") if column == "plot" else column for column in df.columns])
#new_df = df.withColumn("plot",)


genreMap = spark.read.load("/home/cse587/CSE587/Project3/mapping.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
genreMap = genreMap.withColumnRenamed('_c0','id')
genreMap = genreMap.withColumnRenamed('0','genre')

#this converts a column into a list for use inside encodeGenre() 
#since you cant perform computations on a dataframe inside a UDF
genreList = genreMap.select("genre").rdd.flatMap(lambda x: x).collect()


def encodeGenre(s):
    #s : the value of col "genre" at some row
    retVal=""
    #s = re.sub(r'[^\\'']',"",s)
    #print("s: ",s)
    #print("type of s: ",type(s))
    
    #convert from string to list
    s = s[1:-1].split(", ")
    #print("type of s: ",type(s))
    #print(s)
    
    #remove single quotes, regex wasnt working for some reason
    temp=[]
    for x in s:
        #s.remove(x)
        x=x[1:-1]
#         print(x)
#         index = genreMap["0"==x]
#         print(index)
        temp.append(x)
    s=temp
    #print(s)
    
    #create one hot encoding, 
    for genre in genreList:
        if genre in s:
            retVal=retVal+"1"
        else:
            retVal=retVal+"0"
##### apparetnly it aint kosher to perform df operations inside a udf... so this hour of work was for nothing
#     t = genreMap.withColumn("encoding",F.when(genreMap.genre.isin(s),1).otherwise(0))
#     #t.show()
#     retVal = t.select("encoding").rdd.flatMap(lambda x: x).collect()
#     retVal = ''.join(map(str,retVal))
    return retVal
    
udf2 = UserDefinedFunction(lambda x: encodeGenre(x), StringType())
df = df.select(*[udf2(column).alias("genre") if column == "genre" else column for column in df.columns])

#taken from https://github.com/apache/spark/blob/master/examples/src/main/python/ml/tf_idf_example.py
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
wordsData = tokenizer.transform(df)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)

#this will do part 2
# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-44d3ba00f964>:10 

In [12]:
from pyspark.mllib.classification import *

LogisticRegressionWithSGD.train(featurizedData,iterations = 10)

NameError: name 'featurizedData' is not defined